In [2]:
import tensorflow as tf
with tf.Session() as sess:
    res = tf.where(tf.constant([True, False]), 
         tf.constant([1, 1]), 
         tf.constant([2, 2]))
    print(sess.run(res))

D:\Users\lsk\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[1 2]


In [7]:
def smooth_l1_loss(bbox_pred, bbox_targets, bbox_inside_weights, bbox_outside_weights, sigma=1.0, dim=[1]):
    sigma_2 = sigma ** 2
    box_diff = bbox_pred - bbox_targets
    in_box_diff = bbox_inside_weights * box_diff
    abs_in_box_diff = tf.abs(in_box_diff)
    # tf.less 返回 True or False； a<b,返回True， 否则返回False。
    smoothL1_sign = tf.stop_gradient(tf.to_float(tf.less(abs_in_box_diff, 1. / sigma_2)))
    # 实现公式中的条件分支
    in_loss_box = tf.pow(in_box_diff, 2) * (sigma_2 / 2.) * smoothL1_sign + (abs_in_box_diff - (0.5 / sigma_2)) * (1. - smoothL1_sign)
    out_loss_box = bbox_outside_weights * in_loss_box
    loss_box = tf.reduce_mean(tf.reduce_sum(out_loss_box, axis=dim))
    return loss_box

In [4]:
x = tf.Variable(0.)
x_plus_1 = tf.assign_add(x, 1)

with tf.control_dependencies([x_plus_1]):
    y = x

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(5):
        print(sess.run(y))

0.0
0.0
0.0
0.0
0.0


In [6]:
a = tf.Variable(0.)
b = tf.Variable(0.)
c = tf.Variable(0.)

assign_a = a.assign_add(1.)
assign_b = b.assign_sub(1.)
assign_c = c.assign_add(2.)

assign_op = tf.group([assign_a, assign_b, assign_c])
d = a + b + c

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(assign_op)
    sess.run(d) 

In [8]:
cond = tf.Variable(True)
def true_fn():
    return tf.constant(1)

def false_fn():
    return tf.constant(0)

result = tf.cond(cond, true_fn, false_fn)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(result)) 
    sess.run(cond.assign(False))
    print(sess.run(result)) 

1
0


In [9]:
def step(x):
    x = tf.convert_to_tensor(x)
    
    def greater_zero_fn():
        return tf.constant(1)
    def equal_zero_fn():
        return tf.constant(0)
    def less_zero_fn():
        return tf.constant(-1)
    
    less_equal = tf.cond(x < 0, less_zero_fn, equal_zero_fn)
    result = tf.cond(x > 0, greater_zero_fn, lambda: less_equal)
    
    return result

In [10]:
x=step(0)

In [11]:
def step_case(x):
    x = tf.convert_to_tensor(x)
    
    case_greater = (x > 0, lambda: tf.constant(1))
    case_equal = (tf.equal(x, 0), lambda: tf.constant(0))
    case_less = (x < 0, lambda: tf.constant(-1))
    
    return tf.case([case_greater, case_equal, case_less])

In [16]:
# i用于记录训练次数
i = tf.constant(0)
n = tf.constant(10)
# 定义一个变量也用来记录循环次数，每次body执行，就加1
run_times = tf.Variable(0)  

def cond(i, n):
    return i < n

def body(i, n):
    i = i + 1
    # 给var增量赋值
    global run_times
    run_times.assign_add(1)
    return i, n

result_i, result_n = tf.while_loop(cond, body, [i, n])

In [17]:
i = tf.constant(0)
n = tf.constant(10)
# 定义一个变量用来记录循环次数
run_times = tf.Variable(0)  

def cond(i, n):
    return i < n

def body(i, n):
    # 给var增量赋值
    global run_times
    assign_op = run_times.assign_add(1)
    # 添加控制依赖
    with tf.control_dependencies([assign_op]):
        i = i + 1
    return i, n

result_i, result_n = tf.while_loop(cond, body, [i, n])

In [19]:
import tensorflow as tf;    

a = [1,5,3]  

f1 = tf.maximum(a, 3)  
f2 = tf.minimum(a, 3)  

with tf.Session() as sess:  
    print (sess.run(f1))
    print (sess.run(f2))  

[3 5 3]
[1 3 3]


In [21]:
import tensorflow as tf 

def condition(time, output_ta_l): 
    return tf.less(time, 3) 

def body(time, output_ta_l): 
    output_ta_l = output_ta_l.write(time, [2.4, 3.5])
    return time + 1, output_ta_l

time = tf.constant(0) 
output_ta = tf.TensorArray(dtype=tf.float32, size=1, dynamic_size=True)
result = tf.while_loop(condition, body, loop_vars=[time, output_ta]) 
last_time, last_out = result 
final_out = last_out.stack() 

with tf.Session(): 
    print(last_time.eval()) 
    print(final_out.eval())


3
[[2.4 3.5]
 [2.4 3.5]
 [2.4 3.5]]


In [30]:
import tensorflow as tf;    

a = [1,5,3]  
b = [2,6,2]
f1 = tf.less(a, b)   

with tf.Session() as sess:  
    print (sess.run(f1))

[ True  True False]


In [25]:
def step(x):
    x = tf.convert_to_tensor(x)
    def greater_zero_fn():
        return tf.constant(1)
    def equal_zero_fn():
        return tf.constant(0)
    def less_zero_fn():
        return tf.constant(-1)  
    less_equal = tf.cond(x < 0, less_zero_fn, equal_zero_fn)
    result = tf.cond(x > 0, greater_zero_fn, lambda: less_equal)
    
    return result

In [26]:
print(step(50))

Tensor("cond_4/Merge:0", shape=(), dtype=int32)
